In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::485679173747:role/AWSGlueServiceRole-Lab4
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 63210cae-c128-414c-845a-e2463d58462c
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 63210cae-c128-414c-845a-e2463d58462c to get into ready status...
Session 63210cae-c128-414c-845a-e2463d58462c has been created.




- Na célula a seguir, adicione o código necessário para ler o conteúdo do arquivo _nomes.csv_ para um dataframe Spark.



In [2]:
nomes_df = glueContext.create_dynamic_frame_from_options(
    connection_type="s3", format="csv",
    connection_options={"paths": ["s3://julio-gluelab/lab-glue/input/nomes.csv"], "recurse": True},
    format_options={"withHeader": True}
)
nomes_df = nomes_df.toDF()
nomes_df = nomes_df.withColumn("total", nomes_df["total"].cast("int"))
nomes_df = nomes_df.withColumn("ano", nomes_df["ano"].cast("int"))

- Imprima o esquema do dataframe

In [3]:
nomes_df.printSchema()
nomes_df.show(10)

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| Jennifer|   F|54336|1983|
|  Jessica|   F|45278|1983|
|   Amanda|   F|33752|1983|
|   Ashley|   F|33292|1983|
|    Sarah|   F|27228|1983|
|  Melissa|   F|23472|1983|
|   Nicole|   F|22392|1983|
|Stephanie|   F|22323|1983|
|  Heather|   F|20749|1983|
|Elizabeth|   F|19838|1983|
+---------+----+-----+----+
only showing top 10 rows


- Na próxima célula, escreva o código necessário para alterar a caixa dos valores da coluna _nome_ para MAIÚSCULO

In [4]:
import pyspark.sql.functions as func
nomes_upper_df = nomes_df.withColumn("nome", func.upper(func.col("nome")))
nomes_upper_df.show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| JENNIFER|   F|54336|1983|
|  JESSICA|   F|45278|1983|
|   AMANDA|   F|33752|1983|
|   ASHLEY|   F|33292|1983|
|    SARAH|   F|27228|1983|
|  MELISSA|   F|23472|1983|
|   NICOLE|   F|22392|1983|
|STEPHANIE|   F|22323|1983|
|  HEATHER|   F|20749|1983|
|ELIZABETH|   F|19838|1983|
|  CRYSTAL|   F|17904|1983|
|      AMY|   F|17095|1983|
| MICHELLE|   F|16828|1983|
|  TIFFANY|   F|15960|1983|
| KIMBERLY|   F|15374|1983|
|CHRISTINA|   F|15359|1983|
|    AMBER|   F|14886|1983|
|     ERIN|   F|14835|1983|
|  REBECCA|   F|14711|1983|
|   RACHEL|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows


- Apresente a contagem de linhas contidas no dataframe

In [16]:
nomes_df.count()

1825433


- Agora, apresente a contagem de nomes, agrupando os dados do dataframe pelas colunas _ano_ e _sexo_. Ordene os dados de modo que o ano mais recente aparece como primeiro registro do dataframe.

In [17]:
nomes_df.groupBy("ano","sexo").count().orderBy("ano", ascending=False).show()

+----+----+-----+
| ano|sexo|count|
+----+----+-----+
|2014|   F|19067|
|2014|   M|13977|
|2013|   F|19191|
|2013|   M|14012|
|2012|   M|14216|
|2012|   F|19468|
|2011|   F|19540|
|2011|   M|14329|
|2010|   M|14241|
|2010|   F|19800|
|2009|   M|14519|
|2009|   F|20165|
|2008|   F|20439|
|2008|   M|14606|
|2007|   F|20548|
|2007|   M|14383|
|2006|   F|20043|
|2006|   M|14026|
|2005|   F|19175|
|2005|   M|13358|
+----+----+-----+
only showing top 20 rows


- Qual foi o nome feminino com mais registro e em que ano?

In [18]:
nomes_df.select("nome","ano").where("sexo == 'F'").orderBy("total", ascending=False).show(1)

+-----+----+
| nome| ano|
+-----+----+
|Linda|1947|
+-----+----+
only showing top 1 row


- Qual foi o nome masculino com mais registro e em que ano?

In [19]:
nomes_df.select("nome","ano").where("sexo == 'M'").orderBy("total", ascending=False).show(1)

+-----+----+
| nome| ano|
+-----+----+
|James|1947|
+-----+----+
only showing top 1 row


- Apresente o total de registros (masculinos e femininos) para cada ano presente no dataframe. Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

In [20]:
nomes_df.select("nome", "sexo", "ano").groupBy("sexo", "ano").count().orderBy("ano").show(10)

+----+----+-----+
|sexo| ano|count|
+----+----+-----+
|   M|1880| 1058|
|   F|1880|  942|
|   F|1881|  938|
|   M|1881|  997|
|   M|1882| 1099|
|   F|1882| 1028|
|   F|1883| 1054|
|   M|1883| 1030|
|   F|1884| 1172|
|   M|1884| 1125|
+----+----+-----+
only showing top 10 rows


- Escreva o conteúdo do dataframe com os valores de nome em maiúsculo no S3. Atenção aos requisitos:
  - A gravação deve ocorrer no subdiretório _frequencia_registro_nomes_eua_ do path s3://\<BUCKET\>/lab-glue/
  - O formato deve ser JSON
  - O particionamento deverá ser realizado pelas colunas _sexo_ e _ano_ (nesta ordem)

In [15]:
nomes_upper_df.write.partitionBy(["sexo", "ano"]).format("json").mode("overwrite").save("s3://julio-gluelab/lab-glue/frequencia_registro_nomes_eua/")

In [21]:
job.commit()